In [1]:
import pandas as pd

In [2]:
snpcalls_asian = pd.read_table('./data/genome_Hong_ChangBum_v2_Full_20190106202606.txt', 
                               comment='#', names=['name', 'chrom', 'position', 'genotype'], low_memory=False)

In [3]:
snpcalls_asian.head()

,name,chrom,position,genotype
0,rs3094315,1,752566,AA
1,rs3934834,1,1005806,CC
2,rs9442372,1,1018704,GG
3,rs3737728,1,1021415,GG
4,rs11260588,1,1021658,GG


In [ ]:
snpcalls_european = pd.read_table('./data/genome_Lilly_Mendel_Mom__20080505171156.txt', 
                                  comment='#', names=['name', 'chrom', 'position', 'genotype'], low_memory=False)

In [ ]:
snpcalls_european.head()

In [ ]:
snpcalls = pd.merge(
    snpcalls_asian[['name','chrom','position', 'genotype']], snpcalls_european[['name','chrom','position', 'genotype']],
    how='inner', left_on=['name','chrom','position'], right_on=['name','chrom','position'], suffixes=['_asian', '_european'])

snpcalls.head()

In [ ]:
def genotype_ibd (maternal, paternal):
  r = [0,0]
  if len(maternal) == len(paternal):
    if maternal[0] == paternal[0]:
      r[0] += 1
    if len(maternal) >= 2 or len(paternal) >= 2:
      if maternal[1] == paternal[1]:
        r[1] += 1

  ibd = int(r[0]) + int(r[1])
  return ibd

In [ ]:
ibd_cal = pd.DataFrame(
    snpcalls.apply(lambda row:genotype_ibd(row['genotype_asian'],
                                                    row['genotype_european']),axis=1).tolist(),
    index=snpcalls.index,
    columns='IBD'.split())

In [ ]:
ibd_cal.head()

In [ ]:
ibd_cal = pd.concat([snpcalls,ibd_cal],axis=1,join_axes=[snpcalls.index])
ibd_cal.head()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')

In [ ]:
plt.figure()
plt.style.use('ggplot')
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 20

ibd_cal['IBD'].value_counts()

In [ ]:
fig, ax =plt.subplots()
ibd_cal['IBD'].value_counts().plot(ax=ax, kind='bar')

In [ ]:
ibd_cal[['chrom','position','IBD']].to_csv('./data/ibd.txt',index=False)